In [26]:
##### This code is from Updated Testing Reddit - No Con- bias (Fictitious Play)-01092022
##### This code replace the big real datanetwork with small sythetic network
import scipy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import pandas as pd
import scipy.io
import collections
import sys
import scipy.optimize
import os
import os.path
from math import comb
from game import Game, GameResult
from utils import *

current_path = os.getcwd()

    # Fixed initial condition + memeory = 50

## Mathmatic Functions

### 1. Import Network

In [27]:
network_name = 'manual'
# network_name = 'karate'

In [28]:
network = import_network(network_name)
G, s, n = network.init()
nxG = nx.from_numpy_array(G)

There are 34 agents
[[0. 1. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


Process the Network Data

In [29]:
# the Laplacian matrix
L = scipy.sparse.csgraph.laplacian(G, normed=False)
# A = (I + L)^(-1) - Stanford paper theory
A = np.linalg.inv(np.identity(n) + L)
nxG = nx.from_numpy_array(G)

### 2. Equilibrium & Polarization 
$$P(z) = z ^T * z $$


## 5. Innate Op and Game

# Fictitious Play Start !


In [30]:
# Calculate network polarization
calculate_polarization(s, n, A, L)

Innate_polarization:	1.1743186020487546
Equi_polarization:	0.15967526924290512
Difference:		-1.0146433328058495


In [31]:
# Game Parameters
game_rounds = 3
memory = 10
k = 2
experiment = 1

In [32]:
# configure the game
game = Game(s, A, L, k)
# run the game
result = game.run(game_rounds, memory)

Nodes, opinions
[6, 20] (1, 0)
Nodes, opinions
[9, 18] (0.5, 0.5)
min_history [((9, 18), (0.5, 0.5))]
--------------------
Game 1
----------
min_history [((9, 18), (0.5, 0.5))]
max_history [0 0 0 ... 0 0 0]
fla_min_fre [1.]
Max_por 0.187646205555797
Max_por 0.187646205555797
Max_por 0.21992903743062744
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2575167647391274
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.2772607776786449
Max_por 0.27726077767

In [33]:
def save(result: GameResult, k, experiment):
    pd.DataFrame(result.payoff_matrix).to_csv(
        f'results/Payoff-Matrix-k-{k}-experiment-{experiment}.csv')

    # Save the original standard output
    original_stdout = sys.stdout

    with open(f'results/Result-k-{k}-experiment-{experiment}.txt', "w") as f:
        # Change the standard output to the file we created.
        sys.stdout = f

        print('Initial Condition -(agent, opinion, pol)')
        # print(f'Innate op {s}')
        # print(f'Adjacency matrix {G}')
        # print('Selected Nodeset, k_Opinions, Steady-state polarization')
        print(f'Max:\t{result.first_max}')
        print(f'Min:\t{result.first_min}')

        print('_____________________')
        print(f'Max Pol:\t{result.equi_max}')
        print(f'Min Pol:\t{result.equi_min}')

        # MAXimizer's distribution of LAST 100 iteration
        print('Max_distribution_last_100')
        max_l100_fre = result.max_history_last_100/100
        print(max_l100_fre[np.nonzero(max_l100_fre)])
        # print for small network
        # print(max_history_last_100)
        # # Print for Large Network
        print(np.nonzero(max_l100_fre))

        columns = np.nonzero(max_l100_fre)
        columns = list(columns[0])
        for column in columns:
            (k_nodes, opinions) = game.map_action(column)
            print(f'Max Nodes: {k_nodes}\tOpinion: {opinions}')

        print('Max_distribution_all')
        max_fre = result.max_history/result.game_rounds
        print(max_fre[np.nonzero(max_fre)])
        print([np.nonzero(max_fre)])

        # TODO confirm isn't this block the same as the one above
        columns_all = np.nonzero(max_l100_fre)
        columns_all = list(columns_all[0])
        for column in columns_all:
            (k_nodes, opinions) = game.map_action(column)
            print(f'Max Nodes: {k_nodes}\tOpinion: {opinions}')

        # MINimizer's Strategy in the last 100 round
        counter = collections.Counter(result.min_touched_last_100)
        # frequency of all min options in order
        fla_min_fre = np.array(list(counter.values()))/(100)
        print('Min_distribution_last_100')
        print(fla_min_fre)
        print(counter)
        # print(min_touched_last_100)

        # a dictionary include {'min_option': count of this choice}
        counter_1 = collections.Counter(result.min_touched_all)
        # frequency of all min options in order
        fla_min_fre_1 = np.array(list(counter_1.values()))/result.game_rounds
        print('Min_distribution_all')
        print(fla_min_fre_1)
        print(counter_1)
        np.set_printoptions(precision=3)

        # a dictionary include {'min_option': count of this choice}
        counter_a = collections.Counter(result.min_history)
        print(counter_a)

        print(f'min_recent_{memory}_touched')
        print(result.min_touched)
        print(f'max_recent_{memory}_touched')
        print(result.max_touched)

        # Reset the standard output to its original value
        sys.stdout = original_stdout

In [34]:
save(result, k, experiment)